In [283]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates


from QUANTAXIS.QAUtil import DATABASE
from QUANTAXIS.QAUtil import  trade_date_sse
from QUANTAXIS.QAUtil.QADate_trade import (
    QA_util_get_pre_trade_date,
    QA_util_get_next_trade_date,
    QA_util_if_tradetime
)

import ind.alpha191 as a191
from scipy import stats
from sklearn import preprocessing
import statsmodels.api as sm



%load_ext autoreload
%autoreload 2
%aimport tools.Cacher


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [357]:
def pretreate_data(data):
    returns = smpl.get_current_return(data,'close')
    returns.name = 'returns'
    ret_forward = smpl.get_forward_return(data,'close')
    ret_forward.name = 'ret_forward'
    # {'Open', 'cap', 'close', 'high', 'ind', 'low', 'returns', 'volume', 'vwap'}
    data = pd.concat([data, returns, ret_forward], axis=1)
    data = data.assign(vwap=data.amount/(data.volume*100))
    data = smpl.add_marketvalue_industry(data)
    data.rename(columns = {"open":"Open",'totalCapital':'cap','industry':'ind'}, inplace=True)
    
    data.rename(columns = {"open":"Open"}, inplace=True)
    data = smpl.add_report_inds(data,'netAssetsPerShare')
    data['pb']=data['close']/data['netAssetsPerShare']
    return data
    # return pd.concat([data, close_ind, vwap_ind, low_ind, high_ind, volume_ind, adv20_ind, adv40_ind, adv81_ind, co_mixed_ind, oh_mixed_ind, lv_mixed_ind], axis=1)




In [16]:
def assemble_stocks_by_codes(codes):
    files = list(map(lambda x:x+'_train_qfq',l))
    return pd.concat(list(map(lambda file:load_cache(file,cache_type=CACHE_TYPE.STOCK),files))).sort_index(level=0)

l = smpl.get_codes_from_blockname('沪深300', sse='all')
df_all =  assemble_stocks_by_codes(l)

In [358]:
df_treated = pretreate_data(df_all)

In [20]:
# ids = [11,24,38,41,42,47,57,69,80,82,83,88,93,97]
ids = [11,24]

# ids = np.arange(1,101)
# np.random.shuffle(ids)

# def generate_alpha_factors(fun_ids, df=None):
#     import ind.alpha191 as a191
#     from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)
#     from base.JuUnits import excute_for_multidates

# for i in fun_ids:
for i in ids:
    fun_name = 'alpha'+ '0'*(3- int(np.log10(i)+1)) +str(i)
    # indx = excute_for_multidates(df, lambda x: getattr(a191, fun_name)(x) ,level=1)
    indx = excute_for_multidates(df_treated, lambda x: getattr(a191, fun_name)(x) ,level=1)
    indx.name = fun_name
    # save_cache('{}_train'.format(fun_name), indx, cache_type=CACHE_TYPE.FACTOR)
    print(indx)
# worker=6
# task = task_chunk_split(ids, worker)
# results = parallal_task(worker, generate_alpha_factors, task, df=df_treated)


code
000001   -4.506804e+07
000002    1.596329e+07
000063   -3.630724e+07
000066    3.796777e+06
000069    2.500154e+07
              ...     
603882   -1.483158e+05
603899   -2.834211e+06
603986    1.431323e+05
603993   -1.358729e+08
605499   -4.952623e+05
Name: alpha011, Length: 248, dtype: float64
code
000001   -0.406365
000002   -0.211400
000063    0.276363
000066    0.133742
000069    0.348468
            ...   
603882    1.635349
603899    1.838936
603986    0.895157
603993   -0.114700
605499   -1.127755
Name: alpha024, Length: 248, dtype: float64


In [1008]:
# excute_for_multidates(df_treated.sort_index(), lambda x: a191.alpha011(x) ,level=1)
def STD(data, windows):
    return data.rolling(window=windows, min_periods=windows).std()
def MEAN(data, windows):
    return data.rolling(window=windows, min_periods=windows).mean()
def DELTA(data, windows):
    return data.diff(4)
def SEQUENCE(n):
    return np.arange(1,n+1)

def SMA(data,windows,alpha):
    return data.ewm(adjust=False, alpha=float(alpha)/windows, min_periods=windows, ignore_na=False).mean()

def REGBETA(xs, y, n):
    assert len(y)>=n,  'len(y)!>=n !!!'
    def reg(X,Y):
        try:
            if len(Y)>len(X):
                res = sm.OLS(Y[X.index], X).fit().params
            else:
                res = sm.OLS(Y, X).fit().params
        except Exception as e:
            print(e)
            return np.nan
        return res
    return xs.rolling(window=n, min_periods=n).apply(lambda x:reg(x,y))

def COVIANCE(A,B,d):
    se = pd.Series(np.arange(len(A.index)),index=A.index)
    se = se.rolling(5).apply(lambda x: A.iloc[x].cov(B.iloc[x]))
    return se

def CORR(A,B,d):
    se = pd.Series(np.arange(len(A.index)),index=A.index)
    se = se.rolling(5).apply(lambda x: A.iloc[x].corr(B.iloc[x]))
    return se


def alpha191(data, dependencies=['volume', 'low', 'close', 'high'], max_window=25):
    # CORR(MEAN(VOLUME,20),LOW,5)+(HIGH+LOW)/2-CLOSE
    part1 = (data['volume'].rolling(window=20,min_periods=20).mean()).rolling(window=5,min_periods=5).corr(data['low'])
    return (part1 + data['high']*0.5+data['low']*0.5-data['close'])


print(alpha191(df_treated.loc[(slice(None),['000001']),:]).to_list())


[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.8052007002358739, 0.9164656302612819, 0.9682620553033878, 0.944237056053371, 0.8217283838123137, 0.48438931845817024, 0.799750843260524, 0.7424966541118021, 0.7259703389380174, 0.27852442823955315, 0.9114433342593458, 0.8262220152092237, 0.5827230660863272, 0.5073278740763865, -0.10603246054957793, 0.1469058669459571, 0.16929926456621658, -0.2896343012832556, -0.3655903116609984, 0.47657030008390144, 0.47875969540657515, -0.18768201264387763, -0.7999280614325865, -0.2832988375545611, -0.7877311658019126, -0.8792777850085596, -0.8657625194894223, -0.7653799656208697, 0.42735083232542515, -0.383782527046554, -0.8353664547793134, -0.8056660123399846, -0.8678564888555549, -0.9049504974266824, -0.5267568815348582, 0.2789323799456387, 0.7082449857668767, 0.8758548053565995, 0.7653951990252104, -0.5574733066330388, -0.7884979253653412, -0.9089520960533584, -0.361022113088703, 

In [943]:
# df_all.loc[(slice('2017-01-10'),['000001','000002','000063']),:]
# pd.concat(indxes,axis=1).sort_index().loc[(slice('2017-01-15'),['000001','000002','000063']),:]

# # a = load_cache('alpha11_train', cache_type=CACHE_TYPE.FACTOR)
# # b = load_cache('alpha24_train', cache_type=CACHE_TYPE.FACTOR)
# pd.concat([a,b],axis=1).sort_index().loc[(slice('2017-01-20'),['000001','000002']),:]
# pd.Series(-1,index=df_treated.loc[(slice(None),['000001']),:].index,dtype=np.dtype('int8'))

# a=alpha004(df_treated.loc[(slice(None),['000001']),:])
# a.rolling(window=5, min_periods=5).corr(a).to_list()
# a.to_list()
# a.rolling(window=5, min_periods=5).apply(lambda x:x.corr(a[x.index])).to_list()
# a[0:50].rolling(window=5, min_periods=5).apply(lambda x:print(x.corr(a[x.index])))


# def alpha083(data, dependencies=['high', 'volume'], max_window=5):
#     # (-1*RANK(COVIANCE(RANK(HIGH),RANK(VOLUME),5)))
#     alpha = COVIANCE(data['high'].rank(pct=True),data['volume'].rank(pct=True),5)*-1
#     return alpha

# print(alpha083(df_treated.loc[(slice(None),['000001']),:]).to_list()[0:100])

# import matplotlib.pyplot as plt
# plt.plot(np.arange(100),alpha083(df_treated.loc[(slice(None),['000001']),:]).to_list()[0:100])
# plt.show()
# plt.plot(np.arange(100),alpha090(df_treated.loc[(slice(None),['000001']),:]).to_list()[0:100])
# plt.show()

# import cpuinfo
# if 'ntel' in cpuinfo.get_cpu_info()['brand_raw']:
#     from sklearnex import patch_sklearn
#     patch_sklearn()
    
import akshare
a = akshare.stock_zh_a_spot_em()
a

,序号,代码,名称,最新价,涨跌幅,涨跌额,成交量,成交额,振幅,最高,最低,今开,昨收,量比,换手率,市盈率-动态,市净率
0,1,688372,N伟测,111.66,81.59,50.17,128990.0,1.185218e+09,60.17,117.00,80.00,90.00,61.49,NaN,71.82,43.96,4.23
1,2,001322,N箭牌,17.01,34.15,4.33,474093.0,8.245760e+08,23.97,18.26,15.22,15.22,12.68,NaN,49.07,34.41,3.76
2,3,688291,N金橙子,34.04,27.16,7.27,166547.0,5.443893e+08,24.80,36.96,30.32,30.60,26.77,NaN,71.28,83.40,3.91
3,4,300598,诚迈科技,52.96,20.01,8.83,183473.0,9.075267e+08,21.44,52.96,43.50,43.50,44.13,2.43,11.46,-50.68,7.62
4,5,688273,麦澜德,42.84,20.00,7.14,67376.0,2.763690e+08,15.66,42.84,37.25,38.80,35.70,4.40,30.14,32.76,3.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,300695,兆丰股份,45.76,-2.43,-1.14,14491.0,6.678290e+07,3.92,47.19,45.35,46.24,46.90,2.00,2.04,19.69,1.41
4996,4997,688237,超卓航科,69.02,-2.43,-1.72,7642.0,5.351282e+07,3.65,71.60,69.02,71.40,70.74,0.82,4.25,84.99,2.98
4997,4998,600348,华阳股份,20.03,-2.44,-0.50,507007.0,1.021440e+09,3.65,20.55,19.80,20.15,20.53,0.78,2.11,7.25,2.19
4998,4999,002112,三变科技,7.85,-2.48,-0.20,628756.0,5.021404e+08,8.94,8.45,7.73,8.40,8.05,3.30,24.33,69.70,4.72


In [966]:
# b=akshare.stock_zh_a_tick_163(code="sz003023", trade_date="20221025") 
# c = akshare.stock_zh_a_tick_tx_js(code="sz003023")
# c['成交时间']
# b['时间']


0       09:25:03
1       09:30:01
2       09:30:08
3       09:30:16
4       09:30:20
          ...   
3705    14:56:54
3706    14:56:57
3707    14:57:00
3708    14:57:03
3709    15:00:03
Name: 时间, Length: 3710, dtype: object